In [2]:
import pandas as pd
import numpy as np

def fibonacci(n):
    fn = [0, 1,]
    for i in range(2, n+1):
        fn.append(fn[i-1] + fn[i-2])
    return fn


N = np.arange(16)
data = {'n': N, 'Fibonacci(n)': fibonacci(15)}
df = pd.DataFrame(data)

In [3]:
def fib_search(f, a, b, n):
    F = fibonacci(n)
    L0 = b - a # Initial interval of uncertainty
    R1 = L0
    Li = (F[n-2]/F[n])*L0

    R = [Li/L0]

    for i in range(2, n+1):
        if Li > L0/2:
            x1 = b - Li
            x2 = a + Li
        else:
            x1 = a + Li
            x2 = b - Li

        f1, f2 = f(x1), f(x2)

        if f1 < f2:
            b = x2
            Li = (F[n - i]/F[n - (i - 2)])*L0 # New interval of uncertainty
        elif f1 > f2:
            a = x1
            Li = (F[n - i]/F[n - (i - 2)])*L0 # New interval of uncertainty
        else:
            a, b = x1, x2
            Li = (F[n - i]/F[n - (i - 2)])*(b - a) # New interval of uncertainty

        L0 = b - a
        R += [Li/R1,]

    if f1 <= f2:
        return [x1, f(x1),R]
    else:
        return [x2, f(x2),R]

In [4]:
def f(x):
    return x**2

interval = fib_search(f,-5,15,7)
min = (interval[0]+interval[1])/2
print(f"[{interval[0]}, {interval[1]}]")
print(min)

[-0.3846153846153839, 0.1479289940828397]
-0.11834319526627211


In [5]:
import math

def golden_section_search(f, a, b, tol=1e-5, max_iter=100):
    r = (math.sqrt(5) - 1) / 2  # golden ratio

    x1 = b - r * (b - a)
    x2 = a + r * (b - a)
    f1 = f(x1)
    f2 = f(x2)

    iter_count = 0
    while abs(b - a) > tol :#and iter_count < max_iter:
        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = b - r * (b - a)
            f1 = f(x1)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = a + r * (b - a)
            f2 = f(x2)
        iter_count += 1

    x_min = (a + b) / 2
    return x_min, f(x_min), iter_count


In [6]:
x_min, f_min, iters = golden_section_search(f, a=0, b=5)
print(f"Minimum at x = {x_min}, f(x) = {f_min}, iterations = {iters}")

Minimum at x = 3.5179209931301263e-06, f(x) = 1.2375768113905654e-11, iterations = 28


In [7]:
from scipy.optimize import minimize_scalar
result = minimize_scalar(f, method='golden', tol=10**-5)
print(result)

 message: 
          Optimization terminated successfully;
          The returned value satisfies the termination criteria
          (using xtol = 1e-05 )
 success: True
     fun: 0.0
       x: 1.57172245561641e-162
     nit: 799
    nfev: 804


### Steepest Descent method

In [8]:
from sympy import symbols, Matrix, zeros, lambdify, diff,Poly
import sympy

def steepest_descent(f, x_vals, learning_rate=0.01, max_iter=1000, tol=1e-6):
    n = len(x_vals)
    x_syms = symbols(f'x1:{n+1}')

    # Convert the symbolic function to a callable numerical function
    f_numeric = lambdify(x_syms, f, 'numpy')

    # Gradient (vector of first partials)
    grad_f_sym = Matrix([diff(f, var) for var in x_syms])
    grad_f_numeric = lambdify(x_syms, grad_f_sym, 'numpy')

    x_vec = Matrix(x_vals).reshape(n, 1)

    for i in range(max_iter):
        # Evaluate gradient at current point
        x_input = [float(x_vec[i, 0]) for i in range(n)]
        grad_val = grad_f_numeric(*x_input) # Use splat operator for argument unpacking

        # Convert numpy array gradient to sympy Matrix for consistency
        grad_val_matrix = Matrix(grad_val).reshape(n, 1)

        # Convergence check
        if grad_val_matrix.norm() < tol:
            print(f"Converged after {i} iterations.")
            break

        # Determine step size (using a fixed learning rate for simplicity here,
        #  a proper line search would be better for general functions)
        alpha = learning_rate

        # Update step
        x_new_vec = x_vec - alpha * grad_val_matrix

        # Update x_vec
        x_vec = x_new_vec
    else:
        print(f"Did not converge after {max_iter} iterations.")


    return x_vec

In [9]:
x1, x2 = symbols('x1 x2 ')
f = x1**2 + 2*x2 + x2**2 + 4

result = steepest_descent(f, [2,1])
print("Minimum at:", result)

Converged after 770 iterations.
Minimum at: Matrix([[3.50839334784554e-7], [-0.999999649160665]])


### Newton's Method

In [10]:
def newton_method(f,x_vals):
    n = len(x_vals)
    x_syms = symbols(f'x1:{n+1}')

    poly = Poly(f,*x_syms)
    terms = poly.terms()

    # Gradient (vector of first partials)
    grad_f = Matrix([diff(f, var) for var in x_syms])

    # Hessian (matrix of second partials)
    H_f = Matrix(n, n, lambda i, j: diff(f, x_syms[i], x_syms[j]))


    x_vec = Matrix(x_vals).reshape(n, 1)
    while True:
        # Evaluate gradient and Hessian at current point by substituting the variables with values
        x_input = [x_vec[i, 0] for i in range(n)]

        # Evaluate gradient and Hessian at current point
        grad_val = grad_f.subs(dict(zip(x_syms, x_vec)))
        hess_val = H_f.subs(dict(zip(x_syms, x_vec)))

        # Convergence check
        if grad_val == zeros(n,1):
            break

        # Newton step: x_new = x - H⁻¹ * grad
        try:
            delta = hess_val.inv() * grad_val
        except:
            print("Hessian is singular. Stopping.")
            break

        x_vec = (x_vec - delta)

    return x_vec



In [11]:
x1, x2, x3 = symbols('x1 x2 x3')
f = x1**4+x1**3-x1+x2**4-x2**2+x2+x3**2-x3+x1*x2*x3

result = newton_method(f, [1, -1, 1])
result_steepest= steepest_descent(f, [1, -1, 1])
print(result)

Hessian is singular. Stopping.
Converged after 737 iterations.


ValueError: Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit